# Simulated streamflow analysis
## Upper Olivares catchment - january, 2023
#### Paul Sandoval Quilodrán - https://github.com/SQPaul/Olivares

#### import packages

In [1]:
import pandas as pd
import numpy as np
import openpyxl
import re
import matplotlib.pyplot as plt
import datetime
import hydroeval as he

### Sensitivity analysis
#### KGE

In [2]:
#Import paths
#obs_path = "P:\Projects\Olivares\Streamflow\qobs_2005.xlsx"
obs_path = "P:\Projects\Olivares\Streamflow\qobs_v0.xlsx"
sim_path = r"P:\Projects\Olivares\SPHY\output\QTOTSubBasinTSS.tss"
#sim_path = r"P:\Projects\Olivares\Calibration\Server\QTOTSubBasinTSS.tss"

#Read and prepare obs streamflow
obs = pd.read_excel(obs_path,sheet_name="4",engine="openpyxl")
obs = obs.iloc[:,1]

#Read and prepare sim streamflow
sim_read = pd.read_csv(sim_path,index_col=False,skiprows=4)
sim = []
for v in range(len(sim_read)):
    val = sim_read.iloc[v,0].strip()
    val = " ".join(val.split())
    sim.append(val)
sim = pd.DataFrame(sim)
sim.columns = ["name"]
sim = sim["name"].str.split(" ",expand=True)
sim = pd.DataFrame(sim.iloc[:,2]).astype(float)
sim = sim.iloc[:,0]

#Calculate KGE'
kge = pd.DataFrame(he.evaluator(he.kgeprime, sim, obs),index=["KGE","r","γ","β"])
kge

,0
KGE,0.640111
r,0.664751
γ,1.054165
β,0.880861


#### NASH

In [3]:
#Import paths
#obs_path = "P:\Projects\Olivares\Streamflow\qobs_2005.xlsx"
obs_path = "P:\Projects\Olivares\Streamflow\qobs_v0.xlsx"
sim_path = r"P:\Projects\Olivares\SPHY\output\QTOTSubBasinTSS.tss"
#sim_path = r"P:\Projects\Olivares\Calibration\Server\QTOTSubBasinTSS.tss"
#thickness_path = r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\output\ICE_DEPTH.csv"


#Read and prepare obs streamflow
obs = pd.read_excel(obs_path,sheet_name="4",engine="openpyxl")
obs = obs.iloc[:,1]

#Read and prepare sim streamflow
sim_read = pd.read_csv(sim_path,index_col=False,skiprows=4)
sim = []
for v in range(len(sim_read)):
    val = sim_read.iloc[v,0].strip()
    val = " ".join(val.split())
    sim.append(val)
sim = pd.DataFrame(sim)
sim.columns = ["name"]
sim = sim["name"].str.split(" ",expand=True)
sim = pd.DataFrame(sim.iloc[:,2]).astype(float)
sim = sim.iloc[:,0]

#Calculate KGE'
nash = pd.DataFrame(he.evaluator(he.nse, sim, obs),index=["NASH"])
nash

,0
NASH,0.367109


### Total streamflow

In [4]:
#Define paths

#rain_path = r"P:\Projects\Olivares\Calibration\Server\QRAINSubBasinTSS.tss"
#snow_path = r"P:\Projects\Olivares\Calibration\Server\QSNOWSubBasinTSS.tss"
#glacier_path = r"P:\Projects\Olivares\Calibration\Server\QGLACSubBasinTSS.tss"
#baseflow_path = r"P:\Projects\Olivares\Calibration\Server\QBASFSubBasinTSS.tss"
obs_path = r"P:\Projects\Olivares\Streamflow\qobs_v0.xlsx"

rain_path = r"P:\Projects\Olivares\SPHY\output\QRAINSubBasinTSS.tss"
snow_path = r"P:\Projects\Olivares\SPHY\output\QSNOWSubBasinTSS.tss"
glacier_path = r"P:\Projects\Olivares\SPHY\output\QGLACSubBasinTSS.tss"
baseflow_path = r"P:\Projects\Olivares\SPHY\output\QBASFSubBasinTSS.tss"

paths = [rain_path, snow_path, glacier_path, baseflow_path]

rain = []
snow = []
glacier = []
baseflow = []
sim = []

streamflow = [rain,snow,glacier,baseflow]

for i in range(len(paths)):
    var = pd.read_csv(paths[i],index_col=False,skiprows=4)
    for j in range(len(var)):
        val = var.iloc[j,0].strip()
        val = " ".join(val.split())
        streamflow[i].append(val)       
    streamflow[i] = pd.DataFrame(streamflow[i])
    streamflow[i].columns = ["name"]
    streamflow[i] = streamflow[i]["name"].str.split(" ",expand=True)
    streamflow[i] = pd.DataFrame(streamflow[i].iloc[:,2])


obs = pd.read_excel(obs_path,sheet_name="4",engine="openpyxl")
        
rain = pd.DataFrame(streamflow[0]).astype(float)
rain.columns = ["rain"]
snow = pd.DataFrame(streamflow[1]).astype(float)
snow.columns = ["snow"]
glacier = pd.DataFrame(streamflow[2]).astype(float)
glacier.columns = ["glacier"]
baseflow = pd.DataFrame(streamflow[3]).astype(float)
baseflow.columns = ["baseflow"]

for k in range(len(rain)):
    sim.append(rain.iloc[k,0]+snow.iloc[k,0]+glacier.iloc[k,0]+baseflow.iloc[k,0])

sim = pd.DataFrame(sim)
sim.columns = ["sim"]
obs = pd.DataFrame(obs.iloc[:,1]).astype(float)
obs.columns = ["obs"]

streamflow = pd.concat([baseflow,rain,snow,glacier,sim,obs],axis=1)
streamflow.to_excel(r"P:\Projects\Olivares\Streamflow\qsim.xlsx")